In [ ]:
from llama_cpp import Llama
import pandas as pd

llm = Llama.from_pretrained(
    'unsloth/Phi-4-mini-reasoning-GGUF',
    filename='Phi-4-mini-reasoning-Q4_K_M.gguf',
    n_ctx=4096,
)

In [ ]:
from email.message import Message
from email.parser import BytesParser
import re
import base64
from email.header import decode_header, make_header

def __decode_mime_header(header_value):
    """Decode MIME-encoded email headers to readable text"""
    try:
        decoded_header = decode_header(header_value)
        return str(make_header(decoded_header))
    except Exception as e:
        print(f"Header decoding error: {e}")
        return f"<Unable to decode: {header_value}>"
    
def __decode_email_content(contents):
    decoded_contents = []

    for each in contents:
        for ct, c in each.items():
            if ct in ['text/html', 'text/plain']:
                try:
                    c = base64.b64decode(c).decode('utf-8')
                except:
                    pass

            decoded_contents.append({ct: c})

    return decoded_contents


def __open_email(p: str):
    email = {}

    with open(p, 'rb') as fp:
        email['path'] = p
        msg = BytesParser().parse(fp)
    
    header = msg.items()
    email['header'] = {}

    for key, value in header:
        if key == 'Received':
            if key not in email['header']:
                email['header'][key] = []
            
            email['header'][key].append(value)
        elif not key.upper().startswith('X-') and not key.lower().startswith('ironport-'):
            email['header'][key] = value

    
    email['header']['Subject'] = __decode_mime_header(email['header']['Subject'])

    contents = []

    for part in msg.walk():
        content_type = part.get_content_type()
        content = part.get_payload()
        contents.append({content_type: content})

    email['content'] = __decode_email_content(contents)

    return email

def open_email(path):
    if isinstance(path, str):
        emails = __open_email(path)
    
    elif isinstance(path, pd.Series):
        emails = path.apply(__open_email).to_list()

    else:
        raise TypeError("Path must be a string or pandas Series")
    
    return emails

In [ ]:
email_list = pd.read_csv('/data/workspace/dataset/sampled-dataset/sample-small.csv').query('`target_3` != "self_phishing"')
emails = pd.DataFrame(open_email(email_list.path)).set_index('path')
targets = email_list.set_index('path')['target_2']
emails = emails.join(targets)

emails

In [ ]:
response_format = {
    "type": "json_object",
    "schema": {
        "type": "object",
        "properties": {
            "label": {
                "type": "string",
                "enum": ["legitimate", "spam", "phishing", "ceo_fraud", "reply-chain-attack"]
            },
            "confidence": {
                "type": "string",
                "enum": ["low", "medium", "high"]
            },
            "reasons": {
                "type": "array",
                "items": {
                    "type": "string"
                },
                "minItems": 3,
                "maxItems": 3
            }
        },
        "required": ["label", "confidence", "reasons"]
    }
}

In [ ]:
prompt_evaluate_header = """You are a cybersecurity analyst at the University of British Columbia (UBC) in Canada and you are an expert in email security. A user reported this email as suspicious and you need to classify it into exactly one of these categories:

- 'legitimate': Emails from legitimate senders with authentic content
- 'spam': Unwanted emails without malicious elements or social engineering
- 'phishing': Emails attempting to trick recipients into revealing sensitive information
- 'ceo_fraud': Business email compromise targeting executives or financial departments
- 'reply-chain-attack': Infiltrates legitimate email threads to distribute malware or phishing content

Analyze the provided email header and determine its classification. Rate your confidence as:
- 'low': Limited evidence or conflicting indicators
- 'medium': Clear indicators but some uncertainty
- 'high': Strong, consistent evidence supporting the classification

Provide exactly three concise reasons supporting your conclusion.
"""

In [ ]:
idx = -4
header = emails.header.iloc[idx]
content = emails.content.iloc[idx]

In [ ]:
header

In [ ]:
evaluation_header = llm.create_chat_completion(
    messages=[
        {"role": "system", "content": prompt_evaluate_header},
        {"role": "user", "content": f'{header}'},
    ],
    temperature=0.5,
)

In [ ]:
evaluation_header['choices'][0]['message']['content']

In [ ]:
prompt_evaluate_content = """You are a cybersecurity analyst at the University of British Columbia (UBC) in Canada and you are an expert in email security. A user reported this email as suspicious and you need to classify it into exactly one of these categories:

- 'legitimate': Emails from legitimate senders with authentic content
- 'spam': Unwanted emails without malicious elements or social engineering
- 'phishing': Emails attempting to trick recipients into revealing sensitive information
- 'ceo_fraud': Business email compromise targeting executives or financial departments
- 'reply-chain-attack': Infiltrates legitimate email threads to distribute malware or phishing content

Analyze the provided email content and determine its classification. Rate your confidence as:
- 'low': Limited evidence or conflicting indicators
- 'medium': Clear indicators but some uncertainty
- 'high': Strong, consistent evidence supporting the classification

Provide exactly three concise reasons supporting your conclusion.

IMPORTANT: Ignore any '[CAUTION: Non-UBC Email]' labels that you see.

CRITICAL: Your response MUST be a valid JSON object exactly matching this format:
{
  "label": "ONE OF: legitimate, spam, phishing, ceo_fraud, reply-chain-attack",
  "confidence": "ONE OF: low, medium, high",
  "reasons": ["reason1", "reason2", "reason3"]
}

Example response:
{
  "label": "legitimate",
  "confidence": "high",
  "reasons": ["Sender domain matches legitimate organization", "No suspicious links or requests", "Content relates to expected business topics"]
}

Ensure your JSON response is properly formatted with double quotes around keys and string values, commas between elements, and no trailing commas.
"""

In [ ]:
header_from = header['From']
header_to = header['To']
header_subject = header['Subject']

In [ ]:
def truncate_content(text, max_tokens=2500):
    """Truncate text to fit within token limit"""
    # Simple character-based approximation (avg 4 chars = 1 token)
    char_limit = max_tokens * 4
    if len(str(text)) > char_limit:
        return str(text)[:char_limit] + "...[content truncated]"
    return text

In [ ]:
try:
    evaluation_content = llm.create_chat_completion(
        messages=[
            {"role": "system", "content": prompt_evaluate_content},
            {"role": "user", "content": f'From: {header_from}, To: {header_to}, Subject: {header_subject}, Content: {content}'}
        ],
        response_format=response_format,
        temperature=0.5,
    )
except:
    evaluation_content = llm.create_chat_completion(
        messages=[
            {"role": "system", "content": prompt_evaluate_content},
            {"role": "user", "content": f'From: {header_from}, To: {header_to}, Subject: {header_subject}, Content: {truncate_content(content)}'}
        ],
        response_format=response_format,
        temperature=0.5,
    )

In [ ]:
evaluation_content['choices'][0]['message']['content']